In [1]:
import numpy as np #For array managment
import math
import sys
sys.path.append('..')
from Simulation import simulation as sim

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def DKL(l1,l2):
    ###Kullback-Leibler divergence between two arrays, they have to be pmfs (ie histograms that sum to 1)
    return(np.sum([l1[i]*math.log(l1[i]/l2[i]) for i in range(len(l1)) if l1[i]!=0]))

In [3]:
S = 4 #For each move, we have four contributors
state_memory = []

In [4]:
def findWall(x):
    ###Check if the agent is near a wall on square x
    # Returns a list of ints, one for each wall (in Python 1<=>True, 0<=>False)
    res = [0, 0, 0, 0]
    if x % 3 == 0:  # If the agent is near the left-side wall, update the list
        res[0] = 1
    if x % 3 == 2:  # If the agent is near the right-side wall, update the list
        res[2] = 1
    if x // 3 == 0:  # If the agent is near the bottom-side wall, update the list
        res[3] = 1
    if x // 3 == 2:  # If the agent is near the top-side wall, update the list
        res[1] = 1
    return (res)  # Return the list

In [5]:
def return_target(x):
    ###Return the pmf corresponding to the target for square x (the behavior is picked depending on the square)

    if x == 0 or x == 3 or x == 4 or x == 6 or x == 7:  # For these squares, we want to go right
        return ([0.01, 0.01, 0.01, 0.96, 0.01])  # Return the pmf corresponding to moving right

    if x == 1 or x == 2 or x == 5:  # For these squares, we want to go up
        return ([0.01, 0.01, 0.96, 0.01, 0.01])  # Return the pmf corresponding to moving top

    return ([0.96, 0.01, 0.01, 0.01, 0.01])  # Else, we want to stay on position

In [6]:
def return_reward(x):
    ###Return the reward function (voded as a list of its values) for square x
    # Square zero (the starting point) has a reward of -1, and square 4 (the middle) has a reward of -100
    # Again, this function picks the reward based on the square where the agent is
    if x == 1:
        return ([0, -1, -100, 0, 0])
    if x == 3:
        return ([0, 0, 0, -100, -1])
    if x == 5:
        return ([0, -100, 0, 0, 0])
    if x == 7:
        return ([0, 0, 0, 0, -100])
    if x == 4:
        return ([-100, 0, 0, 0, 0])
    if x == 0:
        return ([-1, 0, 0, 0, 0])
    return ([0, 0, 0, 0, 0])

In [7]:
def return_sources(x):
    ###Return the behavior of the sources for square x
    source_left = [0.01, 0.96, 0.01, 0.01, 0.01]
    source_top = [0.01, 0.01, 0.96, 0.01, 0.01]
    source_right = [0.01, 0.01, 0.01, 0.96, 0.01]
    source_bot = [0.01, 0.01, 0.01, 0.01, 0.96]
    source_stay = [0.96, 0.01, 0.01, 0.01, 0.01]

    if findWall(x)[0]:  # If we are near a wall (or more), the corresponding movement is replaced by "stay in place"
        source_left = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[1]:
        source_top = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[2]:
        source_right = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[3]:
        source_bot = [0.96, 0.01, 0.01, 0.01, 0.01]

    return (np.array([source_left, source_top, source_right, source_bot]))

In [8]:
def new_state(position, action):
    if action == 0:
        return(position)
    if action == 1:
        return(position - 1)
    if action == 2:
        return(position + 3)
    if action == 3:
        return(position + 1)
    if action == 4:
        return(position - 3)

In [9]:
def control_algorithm(actual_position):
    #     free_grids = findWall(actual_position) #check if there's no walls
    #     available states = [i for i, x in enumerate(free_grids) if x == 0]
    weights = np.zeros(S)

    target = return_target(actual_position)
    reward = return_reward(actual_position)
    sources = return_sources(actual_position)

    for i in range(S):
        result_DKL = DKL(sources[i], target) - np.dot(sources[i], reward)  # Calculate the weight and store it
        weights[i] = result_DKL

    j = np.argmin(weights)  # Get the minimum weight
    return (return_sources(actual_position)[j])  # Return the corresponding policy

In [10]:
def loop(actual_position):
    ###
    #Simulation loop
    pmf = control_algorithm(actual_position)
    action = np.argmax(pmf) #We use a maximum sampling
    x = new_state(actual_position, action) #Get the corresponding new state
    print('PMF: ' + str(pmf))
    print('New state: ' + str(x))
    return(x)

In [11]:
x = 0
state_memory.append(x)
#Let's do five loops
for i in range(5):
#while x != 8:
    x = loop(x)
    state_memory.append(x)

PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 1
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 2
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 5
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 8
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 8


In [12]:
sim(3, [4], state_memory) #grade_dimension, number of the grid with obstacle
exit()

SystemExit: 

D:\Aplikacje\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
